In [3]:
import numpy as np
import pandas as pd
import math

In [4]:
# Measuring Distance between Patients and Stations

#Plat = Patient Latitude
#Plon = Patient Longitude
#Slat = Station Latitude
#Slon = Station Longitude

def earth_dist(Plat,Plon,Slat,Slon):
    

    rad = math.pi / 180
    
    a1 = Plat * rad
    a2 = Plon * rad
    b1 = Slat * rad
    b2 = Slon * rad
    
    distance_lat = b1 - a1
    distance_lon = b2 - a2
    
    a = math.sin(distance_lat/2)**2 + math.cos(a1) * math.cos(b1) * math.sin(distance_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371.0  # Radius of the Earth in kilometers
    d = R * c
    
    return d

def inverse_distance_weighting(distances, measurement, power=2):
    
    numerator = np.sum(values / (distances ** power))
    weights = np.sum(1 / (distances ** power))
    weighted_measurement = numerator / weights
    
    return weighted_measurement

In [5]:
# Select Stations in postcode_of_station
Path_Station = "C:/Users/Mahin Vazifehdan/Desktop/Datasets/BrainTeaser/AirQuality/postcode_of_station.csv"
postcode_station = pd.read_csv(Path_Station)
postcode_station = postcode_station[postcode_station["country"]==2]
postcode_station.head() # 

,id_postcode_of_station,postcode,country,geom,latitude,longitude
34,610,20096,2,0101000000399D64ABCBBD464048E17A14AEA72240,45.482778,9.327500
35,611,24128,2,0101000000FB928D075BD846401F82AAD1AB492340,45.690278,9.643889
36,612,46100,2,01010000001BEDCBACB8934640872F2610899D2540,45.154073,10.807686
37,613,41124,2,0101000000764F1E166A514640906B43C538CF2540,44.636050,10.904730
38,614,21100,2,0101000000682101A3CBE94640780DFAD2DB9F2140,45.826527,8.812224


In [6]:
# Select patients in Italy with postcode_of_patient
Path_Patient = "C:/Users/Mahin Vazifehdan/Desktop/Datasets/BrainTeaser/AirQuality/postcode_of_patient.csv"
postcode_patient = pd.read_csv(Path_Patient)
postcode_patient = postcode_patient[postcode_patient["country"]==2]
postcode_patient.head()

,id_postcode_of_patient,postcode,country,geom,hash_code,latitude,longitude,matching_post_code,distance
6,14343,72023,2,0101000000736891ED7C474440ADFA5C6DC5CE3140,NaN,40.5585,17.8077,863,0.0
7,14346,72026,2,01010000002FDD240681354440BD5296218ED53140,NaN,40.4180,17.8342,852,0.0
8,14347,72027,2,0101000000D8817346943E44408FC2F5285CFF3140,NaN,40.4889,17.9975,860,0.0
9,14350,72100,2,01010000009BE61DA7E85044401DC9E53FA4EF3140,NaN,40.6321,17.9361,853,0.0
10,14358,76121,2,010100000000917EFB3AA844408048BF7D1D483040,NaN,41.3143,16.2817,980,0.0


In [7]:
Path_Measurement = "C:/Users/Mahin Vazifehdan/Desktop/Datasets/BrainTeaser/AirQuality/measurement_on_postcode_of_station.csv"
airpollution_measurement = pd.read_csv(Path_Measurement)
airpollution_measurement.head()

,id,measurement,postcode,timestamp,param
0,1113600,9.681818,692,2016-03-13 23:00:00,2
1,1113601,7.409091,692,2016-03-14 23:00:00,2
2,1113602,16.041667,692,2016-03-15 23:00:00,2
3,1113603,11.083333,692,2016-03-16 23:00:00,2
4,1113604,11.500000,692,2016-03-17 23:00:00,2


In [196]:
# Finding n nearest stations for each patient
radius = 50 # kilometer
count = 0
result = []
for index_p, row_p in postcode_patient.iterrows():
    temp = []
    for index_s, row_s in postcode_station.iterrows():
        dist = earth_dist(row_p["latitude"],row_p["longitude"],
                          row_s["latitude"],row_s["longitude"])
        
        temp.append([dist, 
                    row_s["id_postcode_of_station"]])
        
    sorted_list = sorted(temp, key=lambda x: x[0])
    n_nearest_stations = [x for x in sorted_list if x[0] <= radius]
    if not n_nearest_stations:
        print("no station within 50 kilometer for postcode number " + row_p["postcode"])
        count = count + 1
    else:
        result.append([row_p["postcode"],
                       row_p["country"],
                       n_nearest_stations])

data = []                     
for p_postcode, p_country, nearest_stations in result:
    for dist, s_id in nearest_stations:
        data.append([p_postcode,
                     p_country,
                     dist,
                     s_id,
                     len(nearest_stations)])

nearest_stations_to_patients = pd.DataFrame(data, columns=["Patient_postcode",
                                                            "Patient_country",
                                                            "Distance(km)",
                                                            "Nearest_station_postcode",
                                                            "Station_Number"])

nearest_stations_to_patients.to_csv("Nearest_Station_Locations.csv", index = False)
nearest_stations_to_patients.head()

no station within 50 kilometer for postcode number 4027
no station within 50 kilometer for postcode number 92013
no station within 50 kilometer for postcode number 98070
no station within 50 kilometer for postcode number 98072
no station within 50 kilometer for postcode number 98075
no station within 50 kilometer for postcode number 98076
no station within 50 kilometer for postcode number 98077
no station within 50 kilometer for postcode number 91022
no station within 50 kilometer for postcode number 57032


,Patient_postcode,Patient_country,Distance(km),Weight,Nearest_station_postcode,Station_Number
0,72023,2,0.786838,1.270910,863,16
1,72023,2,15.098579,0.066231,853,16
2,72023,2,15.382203,0.065010,852,16
3,72023,2,18.587102,0.053801,860,16
4,72023,2,18.923291,0.052845,987,16


In [280]:
# Your existing code to define column_to_param
column_to_param = {
    'PM10': 1,
    'PM25': 2,
    'S02': 3,
    'C6H6': 4,
    'O3': 5,
    'N02': 6,
    'CO': 7
}
no_measurement_count = 0
nearest_data = nearest_stations_to_patients.copy()

# Initialize the result DataFrame
result_data = pd.DataFrame(columns=["postcode_patient", "measurement", "timestamp", "param"])

# Define the range of dates
dates = pd.date_range(start='2012/12/31', end='2021/12/30')
dates = pd.to_datetime(dates)

airpollution_measurement['timestamp'] = pd.to_datetime(airpollution_measurement['timestamp'])


for patient_postcode in nearest_data["Patient_postcode"].unique():

    for param_name, param_value in column_to_param.items():

        for specific_date in dates:
            p_data = nearest_data[nearest_data["Patient_postcode"] == patient_postcode]
            nearest_stations = p_data["Nearest_station_postcode"].unique()

            # Filter rows based on postcode, timestamp, and param
            filtered_rows = airpollution_measurement[
                (airpollution_measurement['postcode'].isin(nearest_stations))
                & (airpollution_measurement['timestamp'].dt.date == specific_date)
                & (airpollution_measurement['param'] == param_value)
            ]
            
            if filtered_rows.empty:
                
                print("No measurement for all nearest neighbors on {} for postcode number {} for param {}"
                      .format(specific_date, patient_postcode, param_name))
            else:
                
                exist_postcode = p_data[p_data["Nearest_station_postcode"].isin(filtered_rows["postcode"])]

                measurment_nearest = filtered_rows.merge(exist_postcode, 
                                                         left_on='postcode', 
                                                         right_on='Nearest_station_postcode')

                measurment_nearest = measurment_nearest[['Patient_postcode', 'Nearest_station_postcode',
                                                         'Distance(km)', 'Weight',
                                                         'measurement', 'timestamp', 'param',
                                                         'Patient_country','Station_Number']]

                distances = measurment_nearest["Distance(km)"]
                measurement = measurment_nearest["measurement"]
                
                weighted_measurement = inverse_distance_weighting(distances, measurement, power=2)
                
                new_row = {
                    "postcode_patient": patient_postcode,
                    "postcode_country" : p_data["Patient_country"].iloc[0],
                    "measurement": weighted_measurement,
                    "timestamp": specific_date,
                    "param": param_value,
                    "Exist_Station_Number": exist_postcode.shape[0],
                    "50km_Station_Number": p_data["Station_Number"].iloc[0]
                }

                result_data = result_data.append(new_row, ignore_index=True)

# Display the result DataFrame
result_data['id'] = range(1,result_data.shape[0]+1)
result_data.shape

No measurement for all nearest neighbors on 2021-07-16 00:00:00 for postcode number 72023 for param PM10
No measurement for all nearest neighbors on 2021-07-17 00:00:00 for postcode number 72023 for param PM10
No measurement for all nearest neighbors on 2021-07-18 00:00:00 for postcode number 72023 for param PM10
No measurement for all nearest neighbors on 2021-07-19 00:00:00 for postcode number 72023 for param PM10
No measurement for all nearest neighbors on 2021-07-20 00:00:00 for postcode number 72023 for param PM10
No measurement for all nearest neighbors on 2021-07-21 00:00:00 for postcode number 72023 for param PM10
No measurement for all nearest neighbors on 2021-07-22 00:00:00 for postcode number 72023 for param PM10
No measurement for all nearest neighbors on 2021-07-23 00:00:00 for postcode number 72023 for param PM10
No measurement for all nearest neighbors on 2021-07-24 00:00:00 for postcode number 72023 for param PM10
No measurement for all nearest neighbors on 2021-12-02 

KeyboardInterrupt: 

In [ ]:
https://gisgeography.com/inverse-distance-weighting-idw-interpolation/
((12/3502) + (10/7502) + (10/8502)) / ((1/3502) + (1/7502) + (1/8502))